In [11]:
from transformers import BertTokenizer , BertModel
import torch

In [2]:
model_name = "models/bert-base-uncased"

In [3]:
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [4]:
test_sentence = "this is a test sentence"

In [5]:
inputs = tokenizer(test_sentence , return_tensors="pt")

In [6]:
inputs

{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3231, 6251,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [7]:
input_ids = inputs["input_ids"]

In [8]:
input_ids

tensor([[ 101, 2023, 2003, 1037, 3231, 6251,  102]])

In [9]:
token_type_ids = inputs["token_type_ids"]

In [10]:
token_type_ids

tensor([[0, 0, 0, 0, 0, 0, 0]])

In [12]:
pos_ids = torch.arange(input_ids.shape[1])

In [13]:
pos_ids

tensor([0, 1, 2, 3, 4, 5, 6])

## 1. token embedding

In [17]:
token_embed = model.embeddings.word_embeddings(input_ids)

In [18]:
token_embed.shape

torch.Size([1, 7, 768])

In [19]:
token_embed

tensor([[[ 1.3630e-02, -2.6490e-02, -2.3503e-02,  ...,  8.6805e-03,
           7.1340e-03,  1.5147e-02],
         [-5.7095e-02,  1.5283e-02, -4.6868e-03,  ..., -3.2484e-03,
           9.7317e-05,  9.4175e-03],
         [-3.6044e-02, -2.4606e-02, -2.5735e-02,  ...,  3.3691e-03,
          -1.8300e-03,  2.6855e-02],
         ...,
         [ 2.3670e-02, -6.1351e-02, -2.9575e-02,  ..., -1.0239e-02,
          -7.2316e-03, -1.1745e-01],
         [ 3.2079e-02,  6.3135e-03, -6.4352e-03,  ..., -1.1689e-03,
          -1.0810e-01, -8.9524e-02],
         [-1.4521e-02, -9.9615e-03,  6.0263e-03,  ..., -2.5035e-02,
           4.6379e-03, -1.5378e-03]]], grad_fn=<EmbeddingBackward0>)

## 2. segment embedding

In [14]:
segment_embed = model.embeddings.token_type_embeddings(token_type_ids)

In [15]:
segment_embed.shape

torch.Size([1, 7, 768])

In [16]:
segment_embed

tensor([[[ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
         [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
         [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
         ...,
         [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
         [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
         [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086]]],
       grad_fn=<EmbeddingBackward0>)

## 3. pos embedding

In [20]:
pos_embed = model.embeddings.position_embeddings(pos_ids)

In [21]:
pos_embed.shape

torch.Size([7, 768])

In [22]:
pos_embed

tensor([[ 1.7505e-02, -2.5631e-02, -3.6642e-02,  ...,  3.3437e-05,
          6.8312e-04,  1.5441e-02],
        [ 7.7580e-03,  2.2613e-03, -1.9444e-02,  ...,  2.8910e-02,
          2.9753e-02, -5.3247e-03],
        [-1.1287e-02, -1.9644e-03, -1.1573e-02,  ...,  1.4908e-02,
          1.8741e-02, -7.3140e-03],
        ...,
        [-5.6087e-03, -1.0445e-02, -7.2288e-03,  ...,  2.0837e-02,
          3.5402e-03,  4.7708e-03],
        [-3.0871e-03, -1.8956e-02, -1.8930e-02,  ...,  7.4045e-03,
          2.0183e-02,  3.4077e-03],
        [ 6.4257e-03, -1.7664e-02, -2.2067e-02,  ...,  6.7531e-04,
          1.1108e-02,  3.7521e-03]], grad_fn=<EmbeddingBackward0>)

In [24]:
embeddings = token_embed + segment_embed + pos_embed

In [25]:
embeddings

tensor([[[ 0.0316, -0.0411, -0.0564,  ...,  0.0021,  0.0044,  0.0219],
         [-0.0489,  0.0285, -0.0204,  ...,  0.0190,  0.0265, -0.0045],
         [-0.0469, -0.0156, -0.0336,  ...,  0.0117,  0.0135,  0.0109],
         ...,
         [ 0.0185, -0.0608, -0.0331,  ...,  0.0040, -0.0071, -0.1213],
         [ 0.0294, -0.0017, -0.0217,  ..., -0.0004, -0.0913, -0.0948],
         [-0.0077, -0.0166, -0.0123,  ..., -0.0310,  0.0124, -0.0064]]],
       grad_fn=<AddBackward0>)

In [26]:
embed = model.embeddings.LayerNorm(embeddings)

In [28]:
model.embeddings.dropout(embed)

tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [-0.6485,  0.6739, -0.0932,  ...,  0.4475,  0.6696,  0.1820],
         [-0.6270, -0.0633, -0.3143,  ...,  0.3427,  0.4636,  0.4594],
         ...,
         [ 0.6010, -0.6970, -0.2001,  ...,  0.2960,  0.2060, -1.7181],
         [ 0.8323,  0.2878,  0.0021,  ...,  0.2628, -1.1310, -1.2708],
         [-0.1481, -0.2948, -0.1690,  ..., -0.5009,  0.2544, -0.0700]]],
       grad_fn=<NativeLayerNormBackward0>)